<a href="https://colab.research.google.com/github/anushachatterjee/Fake-News-Detection-NLP-Project/blob/main/Copy_of_LastDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  2 16:50:39 2019
@author: Nobline
"""

import pandas as pd
import numpy as np

#load training data
bodies = pd.read_csv("https://raw.githubusercontent.com/Ethan-Tseng/AI4ALL-NLP/master/Week_2/train_bodies.csv")
headlines = pd.read_csv("https://raw.githubusercontent.com/Ethan-Tseng/AI4ALL-NLP/master/Week_2/train_stances.csv")

import re
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('punkt')


def clean(doc):
    for i in range(doc.shape[0]):
        #lowercasing
        doc.at[i] = doc.iloc[i].lower()
    print("LOWERCASE")
    #print(doc)
    for i in range(doc.shape[0]):
        #remove punctuation
        doc.at[i] = re.sub(r'([^\s\w])+', '', doc.iloc[i])
    print("REMOVE PUNCT")
    #print(doc)
    for i in range(doc.shape[0]):
        #remove stopwords
        doc.at[i] = remove_stopwords(doc.iloc[i])
    print("REMOVE STOPWORDS")
    #print(doc)
    for i in range(doc.shape[0]):
        #tokenize
        doc.at[i] = word_tokenize(doc.iloc[i])
    print("TOKENIZE")
    #print(doc)
    for i in range(doc.shape[0]):
        #lemmatize
        for j in range(len(doc.iloc[i])):
            doc.iloc[i][j] = lemmatizer.lemmatize(doc.iloc[i][j])
    print("LEMMATIZE")
    #print(doc)

testBodies = pd.read_csv("https://raw.githubusercontent.com/Ethan-Tseng/AI4ALL-NLP/master/Week_1/data/competition_test_bodies.csv")
testHeadlines = pd.read_csv("https://raw.githubusercontent.com/Ethan-Tseng/AI4ALL-NLP/master/Week_1/data/competition_test_stances.csv")

clean(bodies['articleBody'])
clean(headlines['Headline'])
clean(testBodies['articleBody'])
clean(testHeadlines['Headline'])

train_data = pd.merge(bodies, headlines, on='Body ID')
test_data = pd.merge(testBodies, testHeadlines, on='Body ID')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
LOWERCASE
REMOVE PUNCT
REMOVE STOPWORDS
TOKENIZE
LEMMATIZE
LOWERCASE
REMOVE PUNCT
REMOVE STOPWORDS
TOKENIZE
LEMMATIZE
LOWERCASE
REMOVE PUNCT
REMOVE STOPWORDS
TOKENIZE
LEMMATIZE
LOWERCASE
REMOVE PUNCT
REMOVE STOPWORDS
TOKENIZE
LEMMATIZE


In [ ]:
#train_data['combine'] = train_data['articleBody'] + train_data['Headline']
combined = pd.concat([train_data['articleBody'],train_data['Headline']])
testCombined = pd.concat([test_data['articleBody'],test_data['Headline']])

In [ ]:

#extract tfidf vectors of article body and headline
from sklearn.feature_extraction.text import TfidfVectorizer

def default(doc):
    return doc

def tfidf_extractor(c,b,h):
    global vect
    vect = TfidfVectorizer(analyzer = 'word', preprocessor = default, tokenizer=default, token_pattern = None)

    global bodyTFIDF
    global headlineTFIDF
    global combineTFIDF
    combineTFIDF = vect.fit(c)
    bodyTFIDF = vect.transform(b)
    headlineTFIDF = vect.transform(h)
    

    
#main action!
tfidf_extractor(combined, train_data['articleBody'], train_data['Headline'])

In [ ]:
def test_tfidf_extractor(b,h):
    global testBodyTFIDF
    global testHeadlineTFIDF
    testBodyTFIDF = vect.transform(b)
    testHeadlineTFIDF = vect.transform(h)
    

    
#main action!
test_tfidf_extractor(test_data['articleBody'], test_data['Headline'])

In [ ]:
#COMBINE VECTOR
from scipy.sparse import coo_matrix, hstack

def stack_headline_body(matrix):
  return hstack(matrix)
  #print(stackedVector.shape)

stackedVector = stack_headline_body([bodyTFIDF, headlineTFIDF])
testStackedVector = stack_headline_body([testBodyTFIDF, testHeadlineTFIDF])



In [ ]:
#Store Labels
# 0 = unrelated 1 = agree 2 = disagree 3 = discuss
#print(train_data['Stance'])
train_data["labels"] = ''
test_data["labels"] = ''
def makeNumStance(data, stance, newColumn):
  for i in range(len(data)): 
      if stance[i] == "unrelated": 
          newColumn[i] = 0
  
      elif stance[i] == "agree": 
          newColumn[i] = 1
  
      elif stance[i] == "disagree": 
          newColumn[i] = 2

      elif stance[i] == "discuss": 
          newColumn[i] = 3

      else:
        newColumn[i] = 'unk'

  return newColumn
    
train_data['labels'] = makeNumStance(train_data, train_data['Stance'], train_data['labels'])
test_data['labels'] = makeNumStance(test_data, test_data['Stance'], test_data['labels'])

# print(train_data['labels'])
# print(test_data['labels'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py

In [ ]:
# Different way to combine the vectors. If the word is present in headline but not in bodies, then mark it as 1, else mark it as 0
# It's okay if this is a little hard to understand, you can assume that this is a blackbox :)

import scipy

def convert_data_to_type_2(bodies, headlines):
    coo_headlines = headlines.tocoo()    
    for i,j,v in zip(coo_headlines.row, coo_headlines.col, coo_headlines.data):
        if bodies[i,j] == 0:
            headlines[i,j] = 1
        else:
            headlines[i,j] = 0
    return headlines

train_data_type2 = convert_data_to_type_2(bodyTFIDF, headlineTFIDF)
test_data_type2 = convert_data_to_type_2(testBodyTFIDF, testHeadlineTFIDF)

# Normalize the data
from sklearn.preprocessing import Normalizer
DataNormalizer = Normalizer()
DataNormalizer.fit(train_data_type2)
train_data_type2 = DataNormalizer.transform(train_data_type2)
test_data_type2 = DataNormalizer.transform(test_data_type2)

In [ ]:
# Import packages to copy data
import copy
import numpy as np

# Define variables which are easy to understand
train_data_type1 = copy.deepcopy(stackedVector)
test_data_type1 = copy.deepcopy(testStackedVector)

train_data_type2 = copy.deepcopy(train_data_type2)
test_data_type2 = copy.deepcopy(test_data_type2)

# Define the labels
train_labels = np.array(train_data['labels'], dtype=np.int)
test_labels = np.array(test_data['labels'], dtype=np.int)

# Print the shapes of data
print("Shape of train data where we combine vectors is: {}".format(train_data_type1.shape))
print("Shape of test data where we combine vectors is: {}".format(test_data_type1.shape))

print("Shape of train data where we \"subtract\" vectors is: {}".format(train_data_type2.shape))
print("Shape of test data where we \"subtract\" vectors is: {}".format(test_data_type2.shape))

# Print the shapes of the labels
print("Shape of train labels is: {}".format(train_labels.shape))
print("Shape of test labels is: {}".format(test_labels.shape))

Shape of train data where we combine vectors is: (49972, 46822)
Shape of test data where we combine vectors is: (25413, 46822)
Shape of train data where we "subtract" vectors is: (49972, 23411)
Shape of test data where we "subtract" vectors is: (25413, 23411)
Shape of train labels is: (49972,)
Shape of test labels is: (25413,)


In [ ]:
# Split the training data into train and test
from sklearn.model_selection import train_test_split

# Define the train data you want to use
train_data_selected = train_data_type2
test_data_selected = test_data_type2

from scipy.sparse import vstack
train_data_stacked = vstack([train_data_selected, test_data_selected])
train_labels_stacked = np.hstack([train_labels, test_labels])

# Define the test data
train_split, test_split, train_labels_split, test_labels_split = train_test_split(train_data_stacked, train_labels_stacked, test_size=0.2, random_state=42)

In [ ]:
# Compute accuracy using Decision Trees, Nearest Neighbors, SVM, Neural Networks

# Define all the import statements
from sklearn.neighbors import KNeighborsClassifier
# Import decision tree
# Import SVM
from sklearn.neural_network import MLPClassifier

# Import the evaluation metrics
from sklearn.metrics import classification_report

# Define the classifier
classifier = KNeighborsClassifier()
# classifier = MLPClassifier(verbose=True, max_iter=40, early_stopping=True)

# Train the classifier
classifier.fit(train_split, train_labels_split)

# Calculate the accuracy of the classifier
print("The performance is:")
print(classification_report(test_labels_split, classifier.predict(test_split)))


The performance is:


KeyboardInterrupt: ignored

In [ ]:
# # Compute accuracy using Decision Trees, Nearest Neighbors, SVM, Neural Networks

# # Define all the import statements
# from sklearn.neighbors import KNeighborsClassifier
# # Import decision tree
# # Import SVM
# from sklearn.neural_network import MLPClassifier
from sklearn import tree

# # Import the evaluation metrics
# from sklearn.metrics import classification_report


# # Define the classifier
# classifier = KNeighborsClassifier()
# # classifier = MLPClassifier(verbose=True, max_iter=40, early_stopping=True)
classifier = tree.DecisionTreeClassifier(min_samples_leaf=1)

# # Train the classifier
classifier.fit(train_data_selected, train_labels)


# # Calculate the accuracy of the classifier
print("The performance is:")
print(classification_report(test_labels, classifier.predict(test_data_type2)))

# # Calculate the accuracy of the classifier
# print("The performance is:")
# print(classification_report(train_labels, classifier.predict(train_data_type1)))

The performance is:
              precision    recall  f1-score   support

           0       0.94      0.56      0.70     18349
           1       0.14      0.26      0.18      1903
           2       0.02      0.02      0.02       697
           3       0.31      0.71      0.43      4464

    accuracy                           0.55     25413
   macro avg       0.35      0.39      0.33     25413
weighted avg       0.75      0.55      0.60     25413

